In [1]:
from datetime import date
import pandas as pd
import yaml
from sqlalchemy import create_engine, inspect, text

### Database connections

In [ ]:
# Cargar configuración desde el archivo config.yml
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_fuente = config['fuente']
    config_bodega = config['bodega']

# Crear conexiones a las bases de datos de fuente y bodega
url_fuente = f"postgresql://{config_fuente['user']}:{config_fuente['password']}@{config_fuente['host']}:{config_fuente['port']}/{config_fuente['dbname']}"
url_bodega = f"postgresql://{config_bodega['user']}:{config_bodega['password']}@{config_bodega['host']}:{config_bodega['port']}/{config_bodega['dbname']}"

# Crear SQLAlchemy engine
fuente_conn = create_engine(url_fuente)
bodega_conn = create_engine(url_bodega)

# Extract

### SQL Query

In [ ]:
# Definir consulta SQL para extraer datos de mensajeros y su información
query = """
SELECT 
    m.id as mensajero_id,
    m.fecha_entrada,
    m.fecha_salida,
    c.nombre as ciudad_operacion,
    m.activo
FROM clientes_mensajeroaquitoy m
LEFT JOIN ciudad c ON m.ciudad_operacion_id = c.ciudad_id
"""

In [ ]:
# Leer datos de la fuente en un DataFrame de pandas
dim_mensajero = pd.read_sql(query, fuente_conn)

# Transformation

In [ ]:
# Reemplazar valores nulos
dim_mensajero = dim_mensajero.replace({
    None: 'No especificado', 
    '': 'No especificado',
    pd.NaT: None  # Para fechas nulas
})

In [6]:
# Asegurarse de que las fechas estén en el formato correcto
dim_mensajero['fecha_entrada'] = pd.to_datetime(dim_mensajero['fecha_entrada']).dt.date
dim_mensajero['fecha_salida'] = pd.to_datetime(dim_mensajero['fecha_salida']).dt.date

In [7]:
# Agregar fecha de carga
dim_mensajero['saved'] = date.today()

### Verifications

In [8]:
print("\nInformación del DataFrame:")
print(dim_mensajero.info())


Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   mensajero_id      50 non-null     int64         
 1   fecha_entrada     33 non-null     object        
 2   fecha_salida      0 non-null      datetime64[ns]
 3   ciudad_operacion  45 non-null     object        
 4   activo            50 non-null     bool          
 5   saved             50 non-null     object        
dtypes: bool(1), datetime64[ns](1), int64(1), object(3)
memory usage: 2.1+ KB
None


In [ ]:
print("\nMuestra de los datos:")
print(dim_mensajero.head())


Muestra de los datos:
   mensajero_id fecha_entrada fecha_salida ciudad_operacion  activo  \
0             1           NaT          NaT      ACOPI YUMBO    True   
1            42           NaT          NaT             CALI    True   
2            48    2024-07-12          NaT             CALI    True   
3            41           NaT          NaT             CALI    True   
4            13    2021-11-08          NaT            PASTO    True   

        saved  
0  2024-11-09  
1  2024-11-09  
2  2024-11-09  
3  2024-11-09  
4  2024-11-09  


# Load

In [10]:
# Cargar en la bodega
dim_mensajero.to_sql('dim_mensajero', bodega_conn, if_exists='replace', index_label='key_dim_mensajero')

50

In [11]:
# Verificaciones adicionales
print("\nNúmero total de mensajeros:", len(dim_mensajero))


Número total de mensajeros: 50


In [12]:
print("\nMensajeros activos vs inactivos:")
print(dim_mensajero['activo'].value_counts())


Mensajeros activos vs inactivos:
activo
True     49
False     1
Name: count, dtype: int64


In [13]:
# Verificar valores nulos
print("\nVerificar valores nulos:")
print(dim_mensajero.isnull().sum())


Verificar valores nulos:
mensajero_id         0
fecha_entrada       17
fecha_salida        50
ciudad_operacion     5
activo               0
saved                0
dtype: int64
